## Physical parameters of the C12's system

C12's quantum computer is based on optimized spin qubits. The spin qubit is realized from electrons trapped in a double quantum dot suspended on carbon nanotubes (CNTs) embedded in a silicon nanocircuit and microwave cavity.

Different physical parameters influence the behavior of our emulator and, consequentially, our gates. Therefore, they can strongly influence the fidelity of the used gate set.


In [1]:
from c12_callisto_clients.api.client import Request

from c12_callisto_clients.user_configs import UserConfigs
import os
user_auth_token = os.getenv("C12_TOKEN")
configs = UserConfigs.parse_obj({"token" : user_auth_token})
# Create the request instance
# Constructor of the Request class also accepts the verbose parameter, which can be use for more detailed output of the methods.

request = Request(auth_token=user_auth_token, verbose=False)

In [2]:
# Getting the physical parameters of the C12 system
physical_params = request.get_params()

for key, value in physical_params.items():
    print(f"{key} = {value['value']} ({value['unit']})")

B_sym = 0.245 (T)
B_asym = 0.245 (T)
epsilon = 0.0 (V)
half_delta_g = 300e6 (Hz)
omega_b = 14.1e6 (Hz)
omega_t = 19.9e9 (Hz)
omega_s = 12.3e9 (Hz)
lambda_nu_b = 1.5e-05 (Hz)
lambda_nu_t = 5e-08 (Hz)
lambda_nu_s = 2e-07 (Hz)
Q_c = 1e4 (none)
Q_nu_b = 1e2 (none)
Q_nu_t = 1e5 (none)
Q_nu_s = 1e5 (none)
T = 20e-3 (K)
Omega_d = 500e6 (Hz)
power_spect_const = 0.00117 (GHz^2)
time_k = 0.5 (none)
noisy = 1 (none)


Changing the physical parameters and passing them when running the emulation is achievable. By doing that, the emulation will be run using the given set of parameters. You can only pass the parameters you want to change. The other ones are going to have the default value.

It is crucial to notice that changing the physical parameters permanently is impossible! They will have default values until they are changed during the start of the emulation.


In [3]:
# For more information see notebooks 2 & 3
from c12simulator_clients.user_configs import UserConfigs
from c12simulator_clients.qiskit_back.c12sim_provider import C12SimProvider

configs = UserConfigs.parse_obj({"token" : user_auth_token, "verbose": False})
c12_simulator_provider = C12SimProvider(configs)
c12_simulator_backend = c12_simulator_provider.get_backend('c12sim-iswap')

from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)


In [4]:
# To change the physical parameters we need to create a Python dictionary with the key (parm name) and value (param value) pairs

new_params = {
    "noisy" : 0
}

c12_job = c12_simulator_backend.run(circuit, shots=10000, physical_params=str(new_params))
c12_result = c12_job.result()
c12_counts = c12_result.get_counts()
print(f"C12 simulation counts: {c12_counts}")

C12 simulation counts: {'00': 5029, '11': 4971}


In [5]:
# To change the physical parameters we need to create a python dictionary with the key (parm name) and value (param value) pairs

new_params = {
    "noisy" : 1
}

c12_job = c12_simulator_backend.run(circuit, shots=10000, physical_params=str(new_params))
c12_result = c12_job.result()
c12_counts = c12_result.get_counts()
print(f"C12 simulation counts: {c12_counts}")

C12 simulation counts: {'00': 4951, '01': 7, '10': 9, '11': 5033}


In [8]:
# Also, it  is possible that some given parameters are outside the ranges that are making them physically impossible
from c12simulator_clients.qiskit_back.exceptions import C12SimJobError
new_params = {
    "Omega_d" : 600e10
}

c12_job = c12_simulator_backend.run(circuit, shots=10000, physical_params=str(new_params))
try:
    c12_result = c12_job.result()
    c12_counts = c12_result.get_counts()
    print(f"C12 simulation counts: {c12_counts}")
except C12SimJobError:
    print(c12_job.error_message()) # Error number 1010 is because the values for the parameters are outside the range


C12SimJobError: Unable to retrieve result for job a7fde476-a75c-454f-9837-7bc2cd225c9c. Job finished with an error state. Use error_message() method to get more details.

In [9]:
print(c12_job.error_message())

Error: 1007 
